In [183]:
from IPython.core.display import HTML
HTML("<style>" + open("style.css").read() + "</style>")

<div class="headline">
Language Technology / Sprachtechnologie
<br><br>
Wintersemester 2019/2020
</div>
<br>
<div class="description">
    Übung zum Thema <i id="topic">"Essay Scoring"</i>
    <br><br>
    Deadline Abgabe: <i #id="submission">Thursday, 05.12.2019 (23:55 Uhr)</i>
</div>

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
import pandas as pd
from sklearn import datasets, svm, tree, metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import sys
import numpy as np
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import nltk
import re
import matplotlib.pyplot as plt

## Warm-up

<div class="task_description">
    <i class="task">Task 7.1:</i> <br>
</div>

Which of the following statements are true?

1. Character n-grams are more robust against grammatical errors than token n-grams.
2. Length features are always good in essay scoring.
3. Length features are more susceptible to cheating than n-gram features.
4. Essay scoring is potentially unfair to very creative learners.

<strong style="color: blue">Lösung: </strong>

1. Not necessarily: character n-grams are robust against spelling variation. With grammatical errors, the problem is often not that the word does not exist, but that the wrong form is used.
2. No, they are only a good indicator when essays are written under a time constraint.
3. True. You can influence length easily (repeating the same words over and over). For n-gram features you will probably have to cover some correct content.
4. True. If someone writes something we have never seen in the training data, the algorithm will probably not know what to do with it. If someone writes mainstream, chances are high we can score it more accurately.

## Features and Machine Learning

<div class="task_description">
    <i class="task">Task 7.2:</i> <br>
</div>

<div class="task_description">
   <i class="subtask">7.2.1</i> <i class="l1">L1</i> <br>
</div>
Import the following data file. It contains a number of linguistic features for essay scoring determined on the first prompt of the ASAP data record. 
<br> Output the NrofTokens feature.

In [ ]:
df = pd.read_csv('featureFileAsap1.tsv', sep='\t')

<strong style="color: blue">Lösung: </strong>

In [ ]:
f1 = df['NrofTokens'].values
print(f1)

You can plot the individual features across the different total scores.

In [ ]:
label ='avgNumCharsToken'
plt.legend(prop={'size': 12})
plt.title(label+' vs Scores')
plt.xlabel(label)
plt.ylabel('Density')
data = df[df.outcome == 2]
sns.distplot(data[label], hist = False, kde = True, label='Score 2')
data = df[df.outcome == 4]
sns.distplot(data[label], hist = False, kde = True, label='Score 4')
data = df[df.outcome == 5]
sns.distplot(data[label], hist = False, kde = True, label='Score 5')
data = df[df.outcome == 6]
sns.distplot(data[label], hist = False, kde = True, label='Score 6')
data = df[df.outcome == 7]
sns.distplot(data[label], hist = False, kde = True, label='Score 7')
data = df[df.outcome == 8]
sns.distplot(data[label], hist = False, kde = True, label='Score 8')
data = df[df.outcome == 9]
sns.distplot(data[label], hist = False, kde = True, label='Score 9')
data = df[df.outcome == 10]
sns.distplot(data[label], hist = False, kde = True, label='Score 10')
data = df[df.outcome == 11]
sns.distplot(data[label], hist = False, kde = True, label='Score 11')
data = df[df.outcome == 12]
sns.distplot(data[label], hist = False, kde = True, label='Score 12')

plt.show()

<div class="task_description">
   <i class="subtask">7.2.2</i> <i class="l1">L1</i> <br>
</div>

Compare the plots for the three features, NrofTokens (essay length), avgNumCharsToken (word length) and PronounRatioI (what proportion of pronouns is 'I'). Which feature separates the different essay levels particularly well and which does not?

<strong style="color: blue">Lösung: </strong>

Essay length works particularly well, the pronoun feature does not work well.

<div class="task_description">
   <i class="subtask">7.2.3</i> <i class="l2">L2</i> <br>
</div>

Train a Decision Tree Model on the data with last week's code and evaluate.

<strong style="color: blue">Lösung: </strong>

In [ ]:
x = df.iloc[:, 1:len(df.columns)-1]
y = df.iloc[:, [len(df.columns)-1]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

c_tree = DecisionTreeClassifier(max_depth=4)
c_tree.fit(x_train, y_train)

predicted = list(c_tree.predict(x_test))
gold = list(y_test.loc[:, "outcome"])
print(classification_report(gold,predicted))

<div class="task_description">
    <i class="task">Task 7.3:</i> <br>
</div>
The following code reads the essays from the first prompt of the ASAP dataset and their corresponding scores.

In [ ]:
df = pd.read_csv('prompt1.tsv', sep='\t')
essays = df['text']
y = df['score1'].values

# Essays are randomly split into 75% training data nd 25% test data
essays_train, essays_test, y_train, y_test = train_test_split(essays, y, test_size=0.25, random_state=1000)

We convert the texts into a matrix of token counts based on the words in the training data. <br>
This is a vector that counts for each word in the train data how often it occurs in an essay:

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(essays_train)
X_train = vectorizer.transform(essays_train)
X_test  = vectorizer.transform(essays_test)

<div class="task_description">
   <i class="subtask">7.3.1</i> <i class="l2">L2</i> <br>
</div>

Print the shape of the train and test data. <br> How many essays do we have? How many features does each essay have?

<strong style="color: blue">Lösung: </strong>

In [ ]:
print(X_train.shape)
print(X_test.shape)

1337 training essays, 446 test essays, 13349 features

First we use a standard shallow learning classifier on these features

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
pred = classifier.predict(X_test)
print("Accuracy:", score)

We also print a confusion matrix, with the lines being the true labels:

In [ ]:
print(confusion_matrix(y_test, pred))

Now we build our first neural model. <br> We add two layers after the input: the first one with 10 nodes, the second one with 6 nodes, one per possible class. We print the summary of the model.

In [ ]:
model = Sequential()
input_dim = X_train.shape[1]
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])
model.summary()

We transform the output value to vectors, instead of '3' We would like to have '[0,0,1,0,0,0]'

In [ ]:
y_test_orig = y_test
y_train = np.interp(y_train, (1, 6), (0, 5))
y_test = np.interp(y_test, (1, 6), (0, 5))
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

Now we do the actual training, we train 10 times on the training data.

In [ ]:
model.fit(X_train, y_train,
                     epochs=10,
                     verbose=True,
                     validation_data=(X_test, y_test),
                     batch_size=10)

We evaluate the loss on the training data and on the validation data.

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=True)
print("Testing Accuracy:  {:.4f}".format(accuracy))

<div class="task_description">
   <i class="subtask">7.3.2</i> <i class="l1">L1</i> <br>
</div>

Observe how the performance develops. Can you explain what is going on here?

<strong style="color: blue">Lösung: </strong>

We overfit on the training data, i.e. our training performance gets better, but we fit so specifically on the training data, that our performance on the test data does not improve anymore.

___
We now get the predictions on the test data and compute a confusion matrix.

In [ ]:
pred = model.predict(X_test)
predarray = []
for p in pred:
    predarray.append(p.argmax()+1)
print(confusion_matrix(y_test_orig, predarray))

In the first net, we represented each essays with an feature vector of fixed length, a BOW model. <br>
Now we represent our input as a sequence of tokens, each token is represented by its index.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(essays_train)

X_train = tokenizer.texts_to_sequences(essays_train)
X_test = tokenizer.texts_to_sequences(essays_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index                                                                                                   

With CountVectorizer, we had stacked vectors of word counts, and each vector was the same length (the size of the total corpus vocabulary). With Tokenizer, the resulting vectors equal the length of each text, and the numbers don’t denote counts, but rather correspond to the word values from the dictionary tokenizer.word_index.
 <br> <br>
 ___
 We consider only the first 500 words per essay, if there are less words, we pad the remaining words with 0s.




In [ ]:
maxlen = 500
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

embedding_dim = 50
model2 = Sequential()
model2.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model2.add(layers.Flatten())
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(6, activation='softmax'))
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model2.summary()
history = model2.fit(X_train, y_train,
                    epochs=20,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model2.evaluate(X_train, y_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(X_test, y_test, verbose=True)
print("Testing Accuracy:  {:.4f}".format(accuracy))

pred = model2.predict(X_test)
predarray = []
for p in pred:
    predarray.append(p.argmax()+1)
print(confusion_matrix(y_test_orig, predarray))

In the following variant of the model, we use pretrained word-embeddings.

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


embedding_dim = 100
embedding_matrix = create_embedding_matrix(
     '/path/to/glove/glove.6B.100d.txt',
     tokenizer.word_index, embedding_dim)


model3 = Sequential()
model3.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=True))
model3.add(layers.Flatten())
model3.add(layers.Dense(10, activation='relu'))
model3.add(layers.Dense(6, activation='softmax'))
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model3.summary()
history = model3.fit(X_train, y_train,
                    epochs=20,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model3.evaluate(X_train, y_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model3.evaluate(X_test, y_test, verbose=True)
print("Testing Accuracy:  {:.4f}".format(accuracy))

pred = model3.predict(X_test)
predarray = []
for p in pred:
    predarray.append(p.argmax()+1)
print(confusion_matrix(y_test_orig, predarray))

<div class="task_description">
   <i class="subtask">7.3.3</i> <i class="l2">L2</i> <br>
</div>

Try to play with some of the parameters: Should we use longer or shorter essays lengths? A different number of hidden units? An additional hidden layer?

## Overall evaluation

<div class="task_description">
    <i class="task">Task 7.4:</i>  <br>
</div>

The following code omputes linearly and quadratically weighted kappa (Set weight to „linear“ or „quadratic“).

In [ ]:
cohen_kappa_score(y1, y2, labels=None, weights=None, sample_weight=None) 

Given is the following confusion matrix. Lines are true values. 

In [ ]:
array=[[20, 5, 10],[15, 20, 5],[10, 20, 10]]
df_cm = pd.DataFrame(array, index = [i for i in "012"],
                     columns = [i for i in "012"])
plt.figure(figsize = (4,3))
sns.heatmap(df_cm, annot=True,cmap="Blues")

<div class="task_description">
   <i class="subtask">7.4.1</i> <i class="l1">L1</i> <br>
</div>

Compute the accuracy and the kappa for the confusion matrix above.

<strong style="color: blue">Lösung: </strong>

In [ ]:
gold0 = [0] * 35 + [1] * 40 + [2] * 40
pred0 = [0] * 20 + [1] * 5 + [2] * 10 + [0] * 15 + [1] * 20 + [2] * 5 + [0] * 10 + [1] * 20 + [2] * 10

print(confusion_matrix(gold0, pred0))
print("Accuracy: ",accuracy_score(gold0, pred0, normalize=True, sample_weight=None))
print("Kappa: ",cohen_kappa_score(gold0, pred0, weights='quadratic'))

<div class="task_description">
   <i class="subtask">7.4.2</i> <i class="l1">L1</i> <br>
</div>
Can you find a different confusion matrix, which has the same label distribution in the gold standard, the same accuracy, but a better QWK (Quadratic Weighted Kappa)?

<strong style="color: blue">Lösung: </strong>

In [ ]:
gold1 = [0] * 35 + [1] * 40 + [2] * 40
pred1 = [0] * 20 + [1] * 5 + [2] * 10 + [0] * 15 + [1] * 20 + [2] * 5 + [0] * 0 + [1] * 30 + [2] * 10

print(confusion_matrix(gold1, pred1))
print("Accuracy: ",accuracy_score(gold1, pred1, normalize=True, sample_weight=None))
print("Kappa: ",cohen_kappa_score(gold1, pred1, weights='quadratic'))

A better matrix has less entries in the extreme cases (0-2, 2-0) while having the same number of gold standard instances per class.

<div class="task_description">
   <i class="subtask">7.4.3</i> <i class="l1">L1</i> <br>
</div>

Can you find one with a worse QWK? 

<strong style="color: blue">Lösung: </strong>

In [ ]:
gold2 = [0] * 35 + [1] * 40 + [2] * 40
pred2 = [0] * 20 + [1] * 0 + [2] * 15 + [0] * 15 + [1] * 20 + [2] * 5 + [0] * 10 + [1] * 20 + [2] * 10

print(confusion_matrix(gold2, pred2))
print("Accuracy: ",accuracy_score(gold2, pred2, normalize=True, sample_weight=None))
print("Kappa: ",cohen_kappa_score(gold2, pred2, weights='quadratic'))

# Homework

<div class="task_description">
    <i class="task">7.1.</i> :::10 Homework points:::
</div>


The following code makes a one-hot feature vector out of an essay and classifies it. 

In [ ]:
my_essay = 'this is an example essay.'
X_testing = tokenizer.texts_to_sequences([my_essay])
X_testing = pad_sequences(X_testing, padding='post', maxlen=maxlen)
print(model2.predict(X_testing).argmax()+1)

<div class="task_description">
   <i class="subtask">7.1.1.</i> <br>
</div>

Try to write a text that gets very high scores and a text that receives very low scores.

<div class="task_description">
   <i class="subtask">7.1.2.</i> <br>
</div>

Can you write something that does not make sense to a human but still receives high scores?

<div class="task_description">
   <i class="subtask">7.1.3.</i> <br>
</div>

Can you write something that should be correct and receives low scores?

____
For each task, describe how you proceeded. <br><br><br>
__Remember we are in prompt 1:__ <br>
<br>*More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends.*
<br><br>
*Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.*
<br><br>
You may have a look at the full scoring guidelines for this prompt.